# Tabbed Interface

> Multi-tab interface pattern with automatic routing, state management, and DaisyUI styling

In [ ]:
#| default_exp patterns.tabbed_interface

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from typing import Dict, Any, Optional, Callable, List, Union
from dataclasses import dataclass, field
from fasthtml.common import *
from fastcore.basics import patch

from cjm_fasthtml_interactions.core.context import InteractionContext
from cjm_fasthtml_interactions.core.html_ids import InteractionHtmlIds
from cjm_fasthtml_daisyui.components.navigation.tabs import tabs, tab, tabs_styles
from cjm_fasthtml_tailwind.utilities.spacing import m
from cjm_fasthtml_tailwind.core.base import combine_classes

## Tab Definition

The `Tab` class defines a single tab in a tabbed interface. Each tab has:
- Unique identifier
- Display label
- Optional title (tooltip on hover)
- Render function that generates the UI
- Optional data loader for fetching required data

In [ ]:
#| export
@dataclass
class Tab:
    """Definition of a single tab in a tabbed interface."""
    
    id: str  # Unique tab identifier (used in URLs)
    label: str  # Display label for the tab
    render: Callable[[InteractionContext], Any]  # Function to render tab content
    title: Optional[str] = None  # Optional title/tooltip for the tab
    data_loader: Optional[Callable[[Any], Dict[str, Any]]] = None  # Data loading function
    load_on_demand: bool = True  # Whether to load content only when tab is selected

## TabbedInterface Class

The `TabbedInterface` class manages a multi-tab interface. It:
- Renders tab navigation using DaisyUI radio-based tabs
- Manages tab content loading via HTMX
- Supports direct navigation to specific tabs via URL
- Generates routes automatically
- Supports different tab styles (lift, bordered, boxed)

In [ ]:
#| export
class TabbedInterface:
    """Manage multi-tab interfaces with automatic route generation and HTMX content loading."""
    
    def __init__(
        self,
        interface_id: str,  # Unique identifier for this interface
        tabs_list: List[Tab],  # List of tab definitions
        default_tab: Optional[str] = None,  # Default tab ID (defaults to first tab)
        container_id: str = InteractionHtmlIds.TABBED_INTERFACE_CONTAINER,  # HTML ID for container
        tabs_id: str = InteractionHtmlIds.TABBED_INTERFACE_TABS,  # HTML ID for tabs element
        content_id: str = InteractionHtmlIds.TABBED_INTERFACE_CONTENT,  # HTML ID for content area
        tab_style: Optional[str] = None,  # DaisyUI tab style (lift, bordered, boxed)
        show_on_htmx_only: bool = False  # Whether to show full page layout for non-HTMX requests
    ):
        """Initialize tabbed interface manager."""
        self.interface_id = interface_id
        self.tabs_list = tabs_list
        self.default_tab = default_tab or (tabs_list[0].id if tabs_list else None)
        self.container_id = container_id
        self.tabs_id = tabs_id
        self.content_id = content_id
        self.tab_style = tab_style
        self.show_on_htmx_only = show_on_htmx_only
        
        # Build tab index for quick lookup
        self.tab_index = {tab.id: idx for idx, tab in enumerate(tabs_list)}

## Tab Management Methods

In [ ]:
#| export
@patch
def get_tab(self:TabbedInterface, 
            tab_id: str  # Tab identifier
           ) -> Optional[Tab]:  # Tab object or None
    """Get tab by ID."""
    idx = self.tab_index.get(tab_id)
    if idx is not None:
        return self.tabs_list[idx]
    return None

In [ ]:
#| export
@patch
def get_tab_index(self:TabbedInterface, 
                  tab_id: str  # Tab identifier
                 ) -> Optional[int]:  # Tab index or None
    """Get tab index by ID."""
    return self.tab_index.get(tab_id)

## Context and Rendering Methods

In [ ]:
#| export
@patch
def create_context(self:TabbedInterface, 
                   request: Any,  # FastHTML request object
                   sess: Any,  # FastHTML session object
                   tab: Tab  # Current tab
                  ) -> InteractionContext:  # Interaction context for rendering
    """Create interaction context for a tab."""
    # Load data if tab has data loader
    data = {}
    if tab.data_loader:
        data = tab.data_loader(request)
    
    return InteractionContext(
        state={},  # Tabs typically don't maintain state between switches
        request=request,
        session=sess,
        data=data
    )

In [ ]:
#| export
@patch
def render_tabs(self:TabbedInterface,
                current_tab_id: str,  # Currently active tab ID
                tab_route_func: Callable[[str], str]  # Function to generate tab route
               ) -> FT:  # Tab navigation element
    """Render tab navigation using DaisyUI radio-based tabs."""
    # Determine tab style classes
    tab_classes = [tabs]
    if self.tab_style:
        # Get the style class from tabs_styles
        style_cls = getattr(tabs_styles, self.tab_style, None)
        if style_cls:
            tab_classes.append(style_cls)
    
    tab_classes.append(m.b(6))  # Add bottom margin
    
    # Create radio inputs for each tab
    tab_inputs = []
    for tab_obj in self.tabs_list:
        tab_inputs.append(
            Input(
                type="radio",
                name=f"{self.interface_id}_tab",
                value=tab_obj.id,
                aria_label=tab_obj.label,
                checked=(current_tab_id == tab_obj.id),
                hx_get=tab_route_func(tab_obj.id),
                hx_target=InteractionHtmlIds.as_selector(self.content_id),
                hx_swap="innerHTML",
                hx_push_url="true",
                title=tab_obj.title or tab_obj.label,
                id=InteractionHtmlIds.tab_radio(tab_obj.id),
                cls=str(tab)
            )
        )
    
    return Div(
        *tab_inputs,
        id=self.tabs_id,
        cls=combine_classes(*tab_classes)
    )

In [ ]:
#| export
@patch
def render_tab_content(self:TabbedInterface,
                       tab_obj: Tab,  # Tab to render
                       ctx: InteractionContext  # Interaction context
                      ) -> FT:  # Tab content
    """Render tab content."""
    return tab_obj.render(ctx)

In [ ]:
#| export
@patch
def render_full_interface(self:TabbedInterface,
                         current_tab_id: str,  # Currently active tab ID
                         tab_route_func: Callable[[str], str],  # Function to generate tab route
                         request: Any,  # FastHTML request object
                         sess: Any  # FastHTML session object
                        ) -> FT:  # Complete tabbed interface
    """Render complete tabbed interface with tabs and content area."""
    current_tab = self.get_tab(current_tab_id)
    if not current_tab:
        current_tab = self.get_tab(self.default_tab)
    
    return Div(
        # Tab navigation
        self.render_tabs(current_tab_id, tab_route_func),
        
        # Tab content area
        Div(
            # Load content on demand if enabled
            hx_get=tab_route_func(current_tab_id) if current_tab.load_on_demand else None,
            hx_trigger="load" if current_tab.load_on_demand else None,
            hx_swap="innerHTML" if current_tab.load_on_demand else None,
            id=self.content_id,
            cls=str(m.t(6))
        ) if current_tab.load_on_demand else Div(
            self.render_tab_content(current_tab, self.create_context(request, sess, current_tab)),
            id=self.content_id,
            cls=str(m.t(6))
        ),
        
        id=self.container_id
    )

## Route Generation

In [ ]:
#| export
@patch
def create_router(self:TabbedInterface,
                  prefix: str = ""  # URL prefix for routes (e.g., "/dashboard")
                 ) -> APIRouter:  # APIRouter with generated routes
    """Create FastHTML router with generated routes for this tabbed interface."""
    router = APIRouter(prefix=prefix)

    # Store reference to interface in router for access in route handlers
    router.tabbed_interface = self

    # Index route - show full interface
    @router
    def index(request, sess, tab_name: str = None):
        """Main tabbed interface entry point."""
        from cjm_fasthtml_app_core.core.htmx import is_htmx_request
        
        # Determine current tab
        current_tab_id = tab_name or self.default_tab
        
        # For HTMX requests, return just the tab content if configured to do so
        if self.show_on_htmx_only and is_htmx_request(request):
            tab_obj = self.get_tab(current_tab_id)
            if tab_obj:
                ctx = self.create_context(request, sess, tab_obj)
                return self.render_tab_content(tab_obj, ctx)
        
        # For non-HTMX or when full interface is needed
        return self.render_full_interface(
            current_tab_id=current_tab_id,
            tab_route_func=lambda tid: tab.to(tab_id=tid),
            request=request,
            sess=sess
        )

    # Tab content route - load individual tab content
    @router
    def tab(request, sess, tab_id: str):
        """Load content for a specific tab."""
        from cjm_fasthtml_app_core.core.htmx import is_htmx_request
        
        tab_obj = self.get_tab(tab_id)
        if not tab_obj:
            # Invalid tab, redirect to default
            tab_obj = self.get_tab(self.default_tab)
        
        # For HTMX requests, return just the tab content
        if is_htmx_request(request):
            ctx = self.create_context(request, sess, tab_obj)
            return self.render_tab_content(tab_obj, ctx)
        
        # For direct navigation (refresh or direct URL access), return full interface
        return self.render_full_interface(
            current_tab_id=tab_id,
            tab_route_func=lambda tid: tab.to(tab_id=tid),
            request=request,
            sess=sess
        )

    return router

## Usage Example

Here's a complete example showing how to create a tabbed interface:

In [ ]:
# Example: Simple 3-tab dashboard
from fasthtml.common import Div, H2, P, H3, Ul, Li

# Define render functions for each tab
def render_overview(ctx: InteractionContext):
    """Render overview tab."""
    stats = ctx.get_data("stats", {})
    return Div(
        H2("Overview"),
        P(f"Total items: {stats.get('total', 0)}"),
        P(f"Active items: {stats.get('active', 0)}")
    )

def render_settings(ctx: InteractionContext):
    """Render settings tab."""
    return Div(
        H2("Settings"),
        H3("Configuration Options"),
        Ul(
            Li("Theme: Dark"),
            Li("Language: English"),
            Li("Notifications: Enabled")
        )
    )

def render_help(ctx: InteractionContext):
    """Render help tab."""
    return Div(
        H2("Help"),
        P("Welcome to the help section."),
        P("Find documentation and guides here.")
    )

# Optional data loader for overview tab
def load_overview_data(request):
    """Load statistics for overview tab."""
    return {
        "stats": {
            "total": 42,
            "active": 15
        }
    }

# Create the tabbed interface with lift style
dashboard_tabs = TabbedInterface(
    interface_id="dashboard",
    tabs_list=[
        Tab(
            id="overview",
            label="Overview",
            title="Dashboard Overview",
            render=render_overview,
            data_loader=load_overview_data
        ),
        Tab(
            id="settings",
            label="Settings",
            title="Configuration Settings",
            render=render_settings
        ),
        Tab(
            id="help",
            label="Help",
            title="Help & Documentation",
            render=render_help
        )
    ],
    tab_style="lift"  # Use DaisyUI lift style
)

# Generate router
dashboard_router = dashboard_tabs.create_router(prefix="/dashboard")

# In your FastHTML app, register the router:
# from cjm_fasthtml_app_core.core.routing import register_routes
# register_routes(app, dashboard_router)
#
# Or directly:
# dashboard_router.to_app(app)

In [ ]:
# Test tabbed interface structure
print(f"Interface has {len(dashboard_tabs.tabs_list)} tabs")
print(f"Default tab: {dashboard_tabs.default_tab}")
print(f"Tab IDs: {list(dashboard_tabs.tab_index.keys())}")
print(f"Tab style: {dashboard_tabs.tab_style}")

Interface has 3 tabs
Default tab: overview
Tab IDs: ['overview', 'settings', 'help']
Tab style: lift


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()